In [1]:
import os
import sys

def adding_module_path():
    module_path = os.path.abspath(os.path.sep.join([".."]*3))

    if module_path not in sys.path:
        sys.path.append(module_path)

adding_module_path()

In [2]:
from src.experiments.instances.neural_nets.transformer_with_dense_head import TransformerWithDenseHeadExperiment
from src.data_loading.get_dataset_object_from import get_dataset_all
from src.utils.from_dataset_arrays import from_dataset_dataframe
from src.models.transformer.pooling_strategy import TransformerPoolingStrategySelection, pooling_strategy_dictionary
from src.types.transformer_name import TransformerName
import plotly.express as px

In [3]:
NUMBER_OF_AUTHORS = 6
NUMBER_OF_SENTENCES = 3
NORMALIZED_VALUE = 1000
SEQUENCE = 50
TRAINABLE = False

In [7]:
import itertools

In [10]:
list(itertools.product([1, 2], [3, 5]))

[(1, 3), (1, 5), (2, 3), (2, 5)]

In [4]:
data, paths = get_dataset_all(NUMBER_OF_AUTHORS, NUMBER_OF_SENTENCES)

Creating shorting method with min = 3
Creating lemma method with instance <WordNetLemmatizer>
Loading dataset from=C:\Users\Vojta\Desktop\diploma\data\gutenberg\6Authors\Sentence3\data.csv


In [5]:
data

[<MapDataset shapes: (<unknown>, <unknown>), types: (tf.string, tf.int32)>]

In [6]:
paths

('C:\\Users\\Vojta\\Desktop\\diploma\\data\\gutenberg\\6Authors\\Sentence3\\data.csv',
 'C:\\Users\\Vojta\\Desktop\\diploma\\data\\gutenberg\\6Authors\\Sentence3\\authors.csv')

In [5]:
all_data = from_dataset_dataframe(data[0])

In [ ]:
all_data['le'] = all_data['text'].apply(lambda x: len(x.split(' ')))

In [ ]:
all_data['le'].mean()

In [ ]:
fig = px.histogram(all_data, x='le', color='label')

In [ ]:
fig.show()

# Testing transformer

In [ ]:
from src.experiments.experiment_run_wrapper import ExperimentRunWrapper
from src.experiments.settings import LearningSettings
from src.tokenizers.transformer_tokenizer import TransformerTokenizer
from src.encoder.create_encoder_from_path import create_encoder_from_path
from transformers import TFAutoModel, AutoConfig
import tensorflow as tf
import os
from src.tokenizers.prepare_dataset_from_tokenizer import prepare_dataset_from_tokenizer
from src.models.transformer.bert_pooling_layer import BertPoolingLayer
import time
from src.experiments.experiment_summarization import ExperimentSummarization
from src.types.experiment_summarization_fields import ExperimentSummarizationFields
from src.config.learning_config import TRANSFORMER_EPOCHS
from src.experiments.descriptions.create_description import (
    create_description_for_transformer_with_dense_head
)
import os
from src.config.config import TEXT_COLUMN, LABEL_COLUMN, VALIDATION_SIZE
from src.experiments.experiment_description import ExperimentDescription
from src.types.transformer_pooling import TransformerPooling
from src.types.prediction_model_type import PredictionModelType
from src.types.net_type import NetType
from src.types.embedding_type import EmbeddingType
from src.types.processing_type import PreprocessingType
import time
from src.data_loading.get_dataset_object_from import get_dataset_all
from src.utils.from_dataset_arrays import from_dataset_dataframe
from src.utils.split_dataframe import split_dataframe
from src.utils.normalize_dataframe_to_size import normalize_dataframe_to_size
from src.utils.create_dataset_from_dataframe import create_dataset_from_Xy
import pandas as pd
from src.utils.generate_random_stamp import generator_random_stamp
from src.types.transformer_pooling import TransformerPooling
from src.models.transformer.pooling_strategy import pooling_strategy_dictionary

In [ ]:
model_name = TransformerName..value

In [ ]:
settings = LearningSettings()
settings.epochs = TRANSFORMER_EPOCHS
data_path, authors_path = paths

data_normalized = normalize_dataframe_to_size(all_data, NORMALIZED_VALUE)
X_train, X_test, y_train, y_test = split_dataframe(data_normalized)
df = pd.DataFrame()
df[TEXT_COLUMN] = X_train
df[LABEL_COLUMN] = y_train
X_train, X_val, y_train, y_val = split_dataframe(df, VALIDATION_SIZE)

train_ds = create_dataset_from_Xy(X_train, y_train)
test_ds = create_dataset_from_Xy(X_test, y_test)
val_ds = create_dataset_from_Xy(X_val, y_val)

### Model test 1

In [ ]:
pooling = pooling_strategy_dictionary[TransformerPoolingStrategySelection.Pooler]

In [ ]:
tokenizer = TransformerTokenizer(
    model_name.value, create_encoder_from_path(authors_path), max_len=SEQUENCE
)

In [ ]:
train_ds = prepare_dataset_from_tokenizer(X_train, tokenizer)
test_ds = prepare_dataset_from_tokenizer(X_test, tokenizer)
val_ds = prepare_dataset_from_tokenizer(X_val, tokenizer)

In [ ]:
config = AutoConfig.from_pretrained(model_name, output_hidden_states=True)
transformer = TFAutoModel.from_config(config)

input_ids = tf.keras.layers.Input(shape=(SEQUENCE,), name="input_ids", dtype="int32")
mask = tf.keras.layers.Input(shape=(SEQUENCE,), name="attention_mask", dtype="int32")


embeddings = transformer(input_ids, attention_mask=mask)['last_hidden_state']  # we only keep tensor 0 (last_hidden_state)
#X = BertPoolingLayer()(embeddings, *pooling_strategy)
X = tf.keras.layers.GlobalMaxPool1D()(embeddings)  # reduce tensor dimensionality
X = tf.keras.layers.BatchNormalization()(X)
X = tf.keras.layers.Dense(128, activation='relu')(X)
X = tf.keras.layers.Dropout(0.2)(X)
y = tf.keras.layers.Dense(NUMBER_OF_AUTHORS, activation='softmax', name='outputs')(X)  # adjust based on number of sentiment classes

model = tf.keras.Model(inputs=[input_ids, mask], outputs=y)

model.layers[2].trainable = TRAINABLE
model

In [ ]:
model.compile(
    loss=settings.loss,
    optimizer=settings.optimizer,
    metrics=settings.metric,
)

In [ ]:
model.summary()

In [ ]:
model.fit()

In [ ]:
prediction = model.predict(test_ds)